In [2]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import pycountry
import geopandas


cost_df = pd.read_csv('./data/cost-of-living.csv', header=0)

print(cost_df.columns)
print(cost_df.shape)
cost_df.head()

#rename 'x54 column'
cost_df.rename(columns={'x54': 'Avg_monthly_salary(USD)'}, inplace=True)

#sort country and city alphabetically
cost_df.sort_values(by=['country', 'city'], ascending=True, inplace=True)

#set No. as index
cost_df.set_index('No.', inplace=True)

cost_df

Index(['No.', 'city', 'country', 'Meal', 'Meal_For_2', 'McDonalds_Meal',
       'Domestic_Beer_Rest(0.5L)', 'Imported_Beer_Rest(0.33L)', 'Cappuccino',
       'Coke/Pepsi', 'Water_Rest(0.33L)', 'Milk', 'Bread', 'Rice', 'Eggs',
       'Cheese', 'Chicken_Fillets', 'Beef', 'Apple', 'Bananas', 'Oranges',
       'Tomato', 'Potato', 'Onion', 'Lettuce', 'Water_Market(1.5L)',
       'Wine_Market', 'Domestic_Beer_Market(0.5L)',
       'Imported_Beer_Market(0.33L)', 'Cigarettes', 'x28', 'x29', 'x30', 'x31',
       'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41',
       'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51',
       'x52', 'x53', 'x54', 'x55', 'data_quality'],
      dtype='object')
(4874, 59)


,city,country,Meal,Meal_For_2,McDonalds_Meal,Domestic_Beer_Rest(0.5L),Imported_Beer_Rest(0.33L),Cappuccino,Coke/Pepsi,Water_Rest(0.33L),...,x47,x48,x49,x50,x51,x52,x53,Avg_monthly_salary(USD),x55,data_quality
No.,,,,,,,,,,,,,,,,,,,,,
2743,Gardez,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
814,Herat,Afghanistan,1.67,6.11,2.78,2.22,NaN,0.56,0.39,0.11,...,19.45,61.12,38.89,111.13,77.79,461.58,238.47,150.02,10.30,0
1389,Jalalabad,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,88.90,NaN,NaN,NaN,NaN,NaN,NaN,9.40,0
129,Kabul,Afghanistan,1.67,5.56,3.06,NaN,NaN,0.92,0.22,0.13,...,35.56,107.12,63.90,238.93,144.47,683.44,350.05,192.62,13.88,1
916,Mazar-e Sharif,Afghanistan,1.67,7.73,2.96,NaN,NaN,0.44,0.34,0.15,...,NaN,66.68,33.34,111.13,55.56,277.82,111.13,222.26,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,Karoi,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2702,Masvingo,Zimbabwe,7.00,40.00,6.00,1.30,2.0,2.20,0.72,0.47,...,51.43,60.00,NaN,300.00,150.00,NaN,582.52,NaN,15.00,0
1746,Mutare,Zimbabwe,7.00,40.00,6.00,1.30,2.0,2.20,0.72,0.47,...,51.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.00,0


In [31]:
#create mortgage_df to drop Nan on 'Avg_monthly_salary(USD)'
mortgage_df = cost_df.dropna(subset=['Avg_monthly_salary(USD)'])

#calculate the 'Max_Mortgage_Pmt(USD)' based on 'Avg_monthly_salary(USD)'
mortgage_df['Max_Mortgage_Pmt(USD)'] = mortgage_df[['Avg_monthly_salary(USD)']].apply(lambda x: round((x*.28), 2))

#calculate 'Avg_mortgage_pmt' per country in general by
#add 'Avg_mortgage_pmt' column to mortgage_df
mortgage_df['Avg_mortgage_pmt'] = round(mortgage_df.groupby(by=['country'])[['Max_Mortgage_Pmt(USD)']].transform(np.mean), 2)

#drop city columns
mortgage_df = mortgage_df[['country', 'Avg_monthly_salary(USD)', 'Avg_mortgage_pmt']]

#drop duplicates for country 
mortgage_df.drop_duplicates(subset='country', inplace=True)

#read code.csv
code_df = pd.read_csv('./data/codes.csv', header=0)

#merge code and mortgage_df
mortgage_code_df = mortgage_df.merge(code_df, on='country').drop(columns=['Latitude (average)', 'Longitude (average)'])

mortgage_df

/mnt/c/Users/Owner/Desktop/Data-engineer/Independent-Project/Cost-of-living/venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/mnt/c/Users/Owner/Desktop/Data-engineer/Independent-Project/Cost-of-living/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


,country,Avg_monthly_salary(USD),Avg_mortgage_pmt
No.,,,
814,Afghanistan,150.02,52.72
2986,Albania,257.39,101.18
3455,Algeria,259.98,73.21
545,Andorra,2473.83,784.45
39,Angola,1428.77,400.06
...,...,...,...
956,Venezuela,200.00,54.46
2853,Vietnam,245.81,121.45
1103,Yemen,279.72,123.12


In [5]:
tech_salaries_df = pd.read_csv('./data/tech-salaries.csv', header=0)

print(tech_salaries_df.columns)
print(tech_salaries_df.shape)
tech_salaries_df.head()

#drop Nan value in 'annual_base_pay', 'job_title', 'location_country'
tech_salaries_df.dropna(subset=['annual_base_pay', 'job_title', 'location_country'], inplace=True) 

#calculate monthly mortgage payment per_country and job_title by: groupby()
#add result to main df by: transform()
tech_salaries_df['Payable_mortgage_per_job_per_month'] = round(tech_salaries_df.groupby(by=['location_country', 'job_title'])[['annual_base_pay']].transform(lambda x: (x/12)*.28), 2)

tech_salaries_df['location_country'].sort_values().unique()

Index(['index', 'salary_id', 'employer_name', 'location_name',
       'location_state', 'location_country', 'location_latitude',
       'location_longitude', 'job_title', 'job_title_category',
       'job_title_rank', 'total_experience_years', 'employer_experience_years',
       'annual_base_pay', 'signing_bonus', 'annual_bonus', 'stock_value_bonus',
       'comments', 'submitted_at'],
      dtype='object')
(1655, 19)


array(['AR', 'AU', 'BD', 'BE', 'BG', 'BM', 'BR', 'CA', 'CH', 'CN', 'CU',
       'CY', 'DE', 'DK', 'ES', 'FI', 'FR', 'GE', 'GR', 'HK', 'HU', 'IE',
       'IL', 'IN', 'IS', 'IT', 'JE', 'KE', 'LU', 'MA', 'MC', 'MD', 'MX',
       'NL', 'NO', 'NZ', 'OM', 'PE', 'PH', 'PK', 'PL', 'PR', 'PT', 'PY',
       'RS', 'RU', 'SE', 'SG', 'SI', 'TW', 'US', 'VE', 'VN', 'ZA'],
      dtype=object)